In [ ]:
# === Colab 환경 설정 ===
# 이 셀은 Google Colab에서만 실행됩니다.

import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🌐 Google Colab 환경 감지됨")
    print("필요한 패키지를 설치합니다...")
    
    # 기본 패키지 (Colab에 이미 설치되어 있지만 확인용)
    !pip install -q numpy matplotlib
    
    print("✅ 환경 설정 완료!")
else:
    print("💻 로컬 환경 감지됨 - 설정을 건너뜁니다.")

# 3차시: 최소 핵반응 네트워크로 He-4까지 만들기

## 학습 목표

이 노트북을 완료하면 다음을 할 수 있다:

1. 핵반응에서 **질량수(A)와 전하(Z) 보존법칙**을 사용하여 반응을 검증한다
2. **6개 핵종**과 **8개 반응**으로 구성된 네트워크를 이해한다
3. **D가 "문"인 이유**와 **He-4로 흐름이 모이는 이유**를 설명한다
4. 반응 네트워크 시뮬레이션 결과를 해석한다

---

## 사전 요구사항

- 1차시 완료 (동결, Γ/H, Yp = 2Xn)
- 2차시 완료 (중수소 병목, 스위치 함수)

In [ ]:
# === 환경 설정 ===
# Google Colab에서 실행할 경우 아래 셀을 실행하세요

# !pip install -q otter-grader

import math
import numpy as np
import matplotlib.pyplot as plt

# 한글 폰트 설정 (Colab용)
try:
    import matplotlib.font_manager as fm
    plt.rcParams['font.family'] = 'DejaVu Sans'
except:
    pass

print("✅ 환경 설정 완료!")

In [ ]:
# === Grader 초기화 ===
import otter
grader = otter.Notebook()
print("✅ Grader 초기화 완료!")

---

# 1-2차시 핵심 함수 (독립 실행용)

3차시를 독립적으로 실행하기 위해 1-2차시의 핵심 함수들을 아래에 포함합니다.
**이 셀은 수정하지 마세요!**

In [ ]:
# === 1-2차시 핵심 상수 및 함수 ===
# (3차시 독립 실행을 위해 포함)

# 물리 상수
A_default = 1.32        # t = A / T^2 관계의 상수 [초·MeV^2]
gamma0_default = 1.3    # Γ = γ₀ T^5 관계의 상수 [1/초·MeV^5]
Q_default = 1.293       # 중성자-양성자 질량차 [MeV]
tau_n_default = 879.0   # 중성자 평균 수명 [초]

def t_of_T(T, A=A_default):
    """온도 T에서 시간 t 계산 (토이 모델)"""
    return A / (T * T)

def Gamma_of_T(T, gamma0=gamma0_default):
    """온도 T에서 약한 반응률 Γ 계산"""
    return gamma0 * (T ** 5)

def Xn_eq(T, Q=Q_default):
    """평형 중성자 분율"""
    return 1.0 / (1.0 + math.exp(Q / T))

def exponential_relaxation(x, x_eq, Gamma, dt):
    """지수 완화 (안정적 수치 적분)"""
    return x_eq + (x - x_eq) * math.exp(-Gamma * dt)

def make_T_grid(T_start=10.0, T_nuc=0.07, n_steps=600):
    """온도 그리드 생성 (로그 스케일)"""
    return np.logspace(np.log10(T_start), np.log10(T_nuc), n_steps)

def step_Xn(Xn, T_prev, T_now, A=A_default, gamma0=gamma0_default, Q=Q_default, tau_n=tau_n_default):
    """한 스텝 업데이트 (1차시 통합 함수)"""
    t_prev = t_of_T(T_prev, A)
    t_now = t_of_T(T_now, A)
    dt = t_now - t_prev
    
    T_mid = math.sqrt(T_prev * T_now)
    lam_np = Gamma_of_T(T_mid, gamma0)
    lam_pn = lam_np * math.exp(-Q / T_mid)
    Gamma_wk = lam_np + lam_pn
    Xeq_mid = lam_pn / Gamma_wk
    
    Xn_temp = exponential_relaxation(Xn, Xeq_mid, Gamma_wk, dt)
    Xn_next = Xn_temp * math.exp(-dt / tau_n)
    Xn_next = max(0.0, min(1.0, Xn_next))
    return Xn_next

def run_bbn_simulation(T_start=10.0, T_nuc=0.07, n_steps=600):
    """1차시 BBN 시뮬레이션 실행"""
    T = make_T_grid(T_start=T_start, T_nuc=T_nuc, n_steps=n_steps)
    Xn = Xn_eq(T[0])
    Xn_list = [Xn]
    t_list = [t_of_T(T[0])]
    Xeq_list = [Xn_eq(T[0])]
    
    for i in range(1, len(T)):
        Xn = step_Xn(Xn, float(T[i-1]), float(T[i]))
        Xn_list.append(Xn)
        t_list.append(t_of_T(T[i]))
        Xeq_list.append(Xn_eq(T[i]))
    
    Yp = 2 * Xn_list[-1]
    return {
        "T": T,
        "t": np.array(t_list),
        "Xn": np.array(Xn_list),
        "Xn_eq": np.array(Xeq_list),
        "Yp": float(Yp),
    }

print("✅ 1-2차시 핵심 함수 로드 완료!")

---

# Part G: 개념 이해 — 보존법칙과 네트워크

---

## G0. 1-2차시 복습 (Warm-up)

1-2차시에서 배운 핵심 개념을 복습합니다.

### G0.1 중수소 병목

"중수소 병목이 열린다"는 것은 무엇을 의미하는가?

- (A) 중수소가 모두 붕괴한다
- (B) 중수소가 **살아남기 시작**한다
- (C) 중성자가 모두 소진된다

In [ ]:
# G0.1: 병목 의미
ans_G0_1 = ...  # "A", "B", "C" 중 선택

### G0.2 제한 반응물

He-4 생성에서 "제한 반응물(limiting reactant)"은 무엇인가?

- (A) 양성자
- (B) 중성자
- (C) 광자

In [ ]:
# G0.2: 제한 반응물
ans_G0_2 = ...  # "A", "B", "C" 중 선택

### G0.3 헬륨 질량비

관측된 우주의 헬륨 질량비 $Y_p$는 대략 얼마인가?

**힌트:** BBN 표준 예측값을 기억하세요.

In [ ]:
# G0.3: Yp 값
ans_G0_3 = ...  # 숫자를 입력하세요 (예: 0.25)

In [ ]:
grader.check('qG0', global_env=globals())

---

## G1. 핵종의 표기법

핵종은 두 가지 숫자로 표현한다:

| 기호 | 이름 | 의미 |
|------|------|------|
| **A** | 질량수 | 핵자(양성자+중성자) 총 개수 |
| **Z** | 원자번호 | 양성자 개수 |

예시:
- 양성자 p: A=1, Z=1
- 중성자 n: A=1, Z=0
- He-4: A=4, Z=2

### G1.1 중수소(D)의 질량수

중수소(D)의 질량수 A는?

In [ ]:
# G1.1: D의 질량수
ans_G1_1 = ...  # 숫자 입력

### G1.2 중수소(D)의 원자번호

중수소(D)의 원자번호 Z는?

**힌트:** 중수소는 수소의 동위원소 (양성자 1개 + 중성자 1개)

In [ ]:
# G1.2: D의 원자번호
ans_G1_2 = ...  # 숫자 입력

In [ ]:
grader.check('qG1', global_env=globals())

---

## G2. 보존법칙

핵반응에서는 다음이 보존된다:

1. **질량수(A):** 핵자의 총 개수
2. **전하(Z):** 양성자의 총 개수

### 예시: $p + n \to D + \gamma$

| | 왼쪽 | 오른쪽 |
|--|------|--------|
| A | 1 + 1 = 2 | 2 + 0 = 2 ✓ |
| Z | 1 + 0 = 1 | 1 + 0 = 1 ✓ |

### G2.1 보존 검사 1

반응 $p + n \to D + \gamma$는 A와 Z를 보존하는가?

- True: 보존한다
- False: 보존하지 않는다

In [ ]:
# G2.1: 보존 검사
ans_G2_1 = ...  # True 또는 False

### G2.2 보존 검사 2

가상의 반응 $p + p \to D + \gamma$는 A와 Z를 보존하는가?

| | 왼쪽 | 오른쪽 |
|--|------|--------|
| A | 1 + 1 = 2 | 2 + 0 = 2 |
| Z | 1 + 1 = 2 | 1 + 0 = 1 |

In [ ]:
# G2.2: 가상 반응 보존 검사
ans_G2_2 = ...  # True 또는 False

In [ ]:
grader.check('qG2', global_env=globals())

---

## G3. 반응 네트워크 지도

### 이번 차시의 6개 핵종

| 핵종 | A | Z | 이름 |
|------|---|---|------|
| n | 1 | 0 | 중성자 |
| p | 1 | 1 | 양성자 |
| D | 2 | 1 | 중수소 |
| T | 3 | 1 | 삼중수소 |
| He-3 | 3 | 2 | 헬륨-3 |
| He-4 | 4 | 2 | 헬륨-4 |

### 네트워크 구조

```
p + n → D → (분기) → T → He-4
                  → He-3 → He-4
```

### G3.1 네트워크에서 분기점

네트워크에서 "분기점(branching point)"이 되는 핵종은?

- (A) n (중성자)
- (B) p (양성자)
- (C) D (중수소)
- (D) He-4 (헬륨-4)

In [ ]:
# G3.1: 분기점 핵종
ans_G3_1 = ...  # "A", "B", "C", "D" 중 선택

In [ ]:
grader.check('qG3', global_env=globals())

---

## G4. D가 "문"인 이유

네트워크를 보면 **모든 경로가 D를 통과**한다:

1. p + n → **D**
2. **D** → T → He-4
3. **D** → He-3 → He-4

2차시에서 배운 "중수소 병목"과 연결해 보자.

### G4.1 D가 "문"인 이유

D가 핵합성의 "문(gate)"인 이유로 가장 적절한 것은?

- (A) **병목**이 열려야 D가 쌓이고, 모든 경로가 D를 거치기 때문
- (B) D가 가장 안정하기 때문
- (C) D가 가장 무겁기 때문

In [ ]:
# G4.1: D가 문인 이유
ans_G4_1 = ...  # "A", "B", "C" 중 선택

In [ ]:
grader.check('qG4', global_env=globals())

---

## G5. 왜 He-4가 많이 생성되는가?

### 세 가지 이유

1. **안정성:** He-4의 핵자당 결합에너지가 가장 높다 (7.1 MeV)
2. **네트워크 구조:** 여러 경로가 모두 He-4로 합류한다
3. **제한 반응물:** 중성자가 제한 반응물이므로 모든 중성자가 He-4로 간다

### G5.1 He-4가 많은 이유

He-4가 BBN의 주요 생성물인 이유로 가장 적절한 것은?

- (A) He-4가 가장 무겁기 때문
- (B) **안정성, 네트워크 구조, 중성자가 제한 반응물**이기 때문
- (C) He-4만 만들어지기 때문

In [ ]:
# G5.1: He-4가 많은 이유
ans_G5_1 = ...  # "A", "B", "C" 중 선택

In [ ]:
grader.check('qG5', global_env=globals())

---

# Part H: 미니 코딩 — 반응 데이터 구조

---

## H0. 핵종 사전 정의

각 핵종의 A와 Z를 딕셔너리로 정의합니다.

In [ ]:
# H0: 핵종 사전
# 이미 정의된 것들:
SPECIES = {
    'n':   {'A': 1, 'Z': 0},   # 중성자
    'p':   {'A': 1, 'Z': 1},   # 양성자
    # ★빈칸★: D와 T를 추가하세요
    'D':   {'A': ..., 'Z': ...},   # 중수소 (힌트: A=2, Z=1)
    'T':   {'A': ..., 'Z': ...},   # 삼중수소 (힌트: A=3, Z=1)
    'He3': {'A': 3, 'Z': 2},   # 헬륨-3
    'He4': {'A': 4, 'Z': 2},   # 헬륨-4
}

# 테스트 출력
print("SPECIES 정의 완료:")
for name, props in SPECIES.items():
    print(f"  {name}: A={props['A']}, Z={props['Z']}")

In [ ]:
grader.check('qH0', global_env=globals())

---

## H1. 반응 데이터 구조

반응을 딕셔너리로 표현합니다:

```python
{
    'reactants': {'species1': count, ...},
    'products': {'species2': count, ...}
}
```

In [ ]:
# H1: 반응 예시 - p + n → D + γ
# (광자 γ는 A=0, Z=0이므로 생략)

reaction_example = {
    'reactants': {'p': 1, 'n': 1},
    # ★빈칸★: products를 완성하세요
    'products': {'D': ...}  # 힌트: D가 1개 생성됨
}

print("반응 예시:", reaction_example)

In [ ]:
grader.check('qH1', global_env=globals())

---

## H2. 보존 검사 함수

반응이 A와 Z를 보존하는지 검사하는 함수입니다.

In [ ]:
# === 제공 함수 ===
def check_conservation(reaction, species=SPECIES):
    """
    반응의 A, Z 보존을 검사한다.
    
    Returns:
        bool: 보존되면 True, 아니면 False
    """
    A_left = sum(species[s]['A'] * n for s, n in reaction['reactants'].items())
    A_right = sum(species[s]['A'] * n for s, n in reaction['products'].items())
    
    Z_left = sum(species[s]['Z'] * n for s, n in reaction['reactants'].items())
    Z_right = sum(species[s]['Z'] * n for s, n in reaction['products'].items())
    
    print(f"A: {A_left} → {A_right} {'✓' if A_left == A_right else '✗'}")
    print(f"Z: {Z_left} → {Z_right} {'✓' if Z_left == Z_right else '✗'}")
    
    return A_left == A_right and Z_left == Z_right

# H2: 보존 검사 실행
# ★빈칸★: reaction_example의 보존 검사를 실행하세요
ans_H2_1 = ...  # 힌트: check_conservation(reaction_example)

print(f"\n보존 여부: {ans_H2_1}")

In [ ]:
grader.check('qH2', global_env=globals())

---

## H3. 반응 1: 중수소 생성

$$p + n \to D + \gamma$$

In [ ]:
# H3: 반응 1 정의
# ★빈칸★: 반응을 완성하세요
reaction_1 = {
    'reactants': {'p': ..., 'n': ...},  # 힌트: 각각 1개
    'products': {'D': ...}              # 힌트: D가 1개
}

# 보존 검사
print("반응 1: p + n → D + γ")
check_conservation(reaction_1)

In [ ]:
grader.check('qH3', global_env=globals())

---

## H4. 반응 2-3: D에서 분기

D에서 두 가지 경로로 분기:

- **반응 2:** $D + p \to {}^3\text{He} + \gamma$ (He-3 경로)
- **반응 3:** $D + n \to T + \gamma$ (T 경로)

In [ ]:
# H4: 반응 2, 3 정의
# ★빈칸★: 두 반응을 완성하세요

# 반응 2: D + p → He3 + γ
reaction_2 = {
    'reactants': {'D': 1, 'p': 1},
    'products': {'He3': ...}  # 힌트: He3가 1개
}

# 반응 3: D + n → T + γ
reaction_3 = {
    'reactants': {'D': 1, 'n': 1},
    'products': {'T': ...}  # 힌트: T가 1개
}

# 보존 검사
print("반응 2: D + p → He3 + γ")
check_conservation(reaction_2)
print()
print("반응 3: D + n → T + γ")
check_conservation(reaction_3)

In [ ]:
grader.check('qH4', global_env=globals())

---

# Part I: 네트워크 시뮬레이션

---

## I0. 전체 8반응 네트워크

8개 반응을 모두 포함한 네트워크를 실행합니다.

**참고:** 이 셀은 토이 모델 시뮬레이션입니다. 정밀한 BBN 코드는 훨씬 복잡합니다.

> 🔴 **블랙박스:** 아래 코드의 `rate` 값들 (1.0, 0.5, 2.0 등)은 **토이 모델의 임의 설정값**이다. 실제 반응률은 온도 함수이며, 핵물리 실험과 이론으로 결정된다. 여기서는 네트워크 구조와 흐름을 이해하는 것이 목적이다.

In [ ]:
# === 전체 8반응 정의 ===
REACTIONS = [
    # 1. p + n → D
    {'name': 'p(n,γ)D', 'reactants': {'p': 1, 'n': 1}, 'products': {'D': 1}, 'rate': 1.0},
    # 2. D + p → He3
    {'name': 'D(p,γ)He3', 'reactants': {'D': 1, 'p': 1}, 'products': {'He3': 1}, 'rate': 0.5},
    # 3. D + n → T
    {'name': 'D(n,γ)T', 'reactants': {'D': 1, 'n': 1}, 'products': {'T': 1}, 'rate': 0.5},
    # 4. D + D → He3 + n
    {'name': 'D(D,n)He3', 'reactants': {'D': 2}, 'products': {'He3': 1, 'n': 1}, 'rate': 0.3},
    # 5. D + D → T + p
    {'name': 'D(D,p)T', 'reactants': {'D': 2}, 'products': {'T': 1, 'p': 1}, 'rate': 0.3},
    # 6. T + D → He4 + n
    {'name': 'T(D,n)He4', 'reactants': {'T': 1, 'D': 1}, 'products': {'He4': 1, 'n': 1}, 'rate': 2.0},
    # 7. He3 + D → He4 + p
    {'name': 'He3(D,p)He4', 'reactants': {'He3': 1, 'D': 1}, 'products': {'He4': 1, 'p': 1}, 'rate': 2.0},
    # 8. He3 + n → T + p (교환)
    {'name': 'He3(n,p)T', 'reactants': {'He3': 1, 'n': 1}, 'products': {'T': 1, 'p': 1}, 'rate': 1.0},
]

print(f"총 {len(REACTIONS)}개 반응 정의됨")
for r in REACTIONS:
    print(f"  {r['name']}")

In [ ]:
# === 토이 네트워크 시뮬레이션 (제공 코드) ===
def run_network_simulation(reactions=REACTIONS, n_steps=500, dt=0.01):
    """
    간단한 토이 네트워크 시뮬레이션.
    
    초기조건: n/p ≈ 1/7 (동결 직후), 다른 핵종은 0
    """
    # 초기 조건 (동결 직후, 병목이 열린 상태)
    Y = {
        'n': 0.12,   # 동결된 중성자 비율
        'p': 0.88,   # 양성자 비율
        'D': 0.0,
        'T': 0.0,
        'He3': 0.0,
        'He4': 0.0,
    }
    
    history = {sp: [Y[sp]] for sp in Y}
    time = [0]
    
    for step in range(n_steps):
        dY = {sp: 0.0 for sp in Y}
        
        for rxn in reactions:
            # 반응률 계산 (단순화)
            rate = rxn['rate'] * dt
            for sp, count in rxn['reactants'].items():
                rate *= max(0, Y[sp]) ** count
            
            # 반응물 소비, 생성물 생성
            for sp, count in rxn['reactants'].items():
                dY[sp] -= rate * count
            for sp, count in rxn['products'].items():
                dY[sp] += rate * count
        
        # 업데이트
        for sp in Y:
            Y[sp] = max(0, Y[sp] + dY[sp])
            history[sp].append(Y[sp])
        
        time.append((step + 1) * dt)
    
    return {'time': np.array(time), 'abundances': {sp: np.array(h) for sp, h in history.items()}}

# 시뮬레이션 실행
result = run_network_simulation()
print("✅ 시뮬레이션 완료!")
print(f"\n최종 풍부도:")
for sp, vals in result['abundances'].items():
    print(f"  {sp}: {vals[-1]:.4f}")

In [ ]:
grader.check('qI0', global_env=globals())

---

## I1. 그래프 해석

In [ ]:
# 그래프 그리기
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 그래프 1: 모든 핵종
ax1 = axes[0]
for sp in ['n', 'p', 'D', 'T', 'He3', 'He4']:
    ax1.plot(result['time'], result['abundances'][sp], label=sp, linewidth=2)
ax1.set_xlabel('시간 (임의 단위)')
ax1.set_ylabel('풍부도 Y')
ax1.set_title('핵종별 풍부도 변화')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 그래프 2: 중간물질 확대
ax2 = axes[1]
for sp in ['D', 'T', 'He3']:
    ax2.plot(result['time'], result['abundances'][sp], label=sp, linewidth=2)
ax2.set_xlabel('시간 (임의 단위)')
ax2.set_ylabel('풍부도 Y')
ax2.set_title('중간물질 (D, T, He-3) 확대')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### I1.1 He-4의 변화

시간에 따른 He-4의 풍부도는 어떻게 변하는가?

- (A) 계속 **증가**한다
- (B) 계속 **감소**한다
- (C) 일정하게 유지된다

In [ ]:
# I1.1: He-4 변화
ans_I1_1 = ...  # "A", "B", "C" 중 선택

### I1.2 중간물질의 패턴

D, T, He-3의 풍부도는 어떤 패턴을 보이는가?

- (A) 계속 증가한다
- (B) **올라갔다가 내려간다** (피크 후 감소)
- (C) 계속 감소한다

In [ ]:
# I1.2: 중간물질 패턴
ans_I1_2 = ...  # "A", "B", "C" 중 선택

In [ ]:
grader.check('qI1', global_env=globals())

---

## I2. 중간물질의 역할

### I2.1 D, T, He-3가 "중간물질"인 이유

D, T, He-3가 "중간물질(intermediate)"이라고 불리는 이유는?

- (A) 가장 먼저 생성되기 때문
- (B) 가장 안정하기 때문
- (C) **만들어지자마자 다음 반응으로 소비**되기 때문

In [ ]:
# I2.1: 중간물질 이유
ans_I2_1 = ...  # "A", "B", "C" 중 선택

In [ ]:
grader.check('qI2', global_env=globals())

---

## I3. 민감도 실험

특정 반응률을 바꾸면 결과가 어떻게 변하는지 관찰합니다.

In [ ]:
# 반응률을 2배로 증가시킨 경우
reactions_modified = [r.copy() for r in REACTIONS]
reactions_modified[5]['rate'] = 4.0  # T(D,n)He4 반응률 2배

result_modified = run_network_simulation(reactions=reactions_modified)

# 비교 그래프
plt.figure(figsize=(10, 5))
plt.plot(result['time'], result['abundances']['He4'], 'b-', linewidth=2, label='기본')
plt.plot(result_modified['time'], result_modified['abundances']['He4'], 'r--', linewidth=2, label='T(D,n)He4 x2')
plt.xlabel('시간 (임의 단위)')
plt.ylabel('He-4 풍부도')
plt.title('민감도 실험: T(D,n)He4 반응률 2배')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"기본 최종 He-4: {result['abundances']['He4'][-1]:.4f}")
print(f"수정 최종 He-4: {result_modified['abundances']['He4'][-1]:.4f}")

### I3.1 민감도 해석

반응률을 바꾸면 결과가 변하는 것을 관찰했습니다. 이것은 무엇을 의미하는가?

- (A) 반응률은 결과에 영향을 주지 않는다
- (B) 반응률이 변하면 **결과도 민감하게 변한다**
- (C) 반응률은 보존법칙을 바꾼다

In [ ]:
# I3.1: 민감도 해석
ans_I3_1 = ...  # "A", "B", "C" 중 선택

In [ ]:
grader.check('qI3', global_env=globals())

---

## I4. Exit Ticket — 최종 이해도 점검

### I4.1 He-4가 많은 이유 (최종)

He-4가 BBN의 주요 생성물인 이유로 가장 **완전한** 설명은?

- (A) He-4가 가장 무겁기 때문
- (B) **네트워크 구조(합류), 안정성(결합에너지), 중성자가 제한 반응물**이기 때문
- (C) 약한 상호작용으로만 만들어지기 때문

In [ ]:
# I4.1: He-4가 많은 이유
ans_I4_1 = ...  # "A", "B", "C" 중 선택

### I4.2 OX 문제

"반응식이 보존법칙을 만족해도, 실제 우주에서 반드시 많이 일어나는 것은 아니다."

- True / False?

In [ ]:
# I4.2: OX 문제
ans_I4_2 = ...  # True 또는 False

### I4.3 다음 차시 예고

4차시에서 다룰 주제는?

- (A) 질량수 7 경로 (${}^7\text{Be}$, ${}^7\text{Li}$)와 **리튬-7 문제**
- (B) 철(Fe)까지의 원소 합성
- (C) 별 내부의 핵융합

In [ ]:
# I4.3: 다음 차시
ans_I4_3 = ...  # "A", "B", "C" 중 선택

In [ ]:
grader.check('qI4', global_env=globals())

---

## ⚠️ 오해하기 쉬운 개념

### 오해 1: "보존법칙만 만족하면 반응이 일어난다"

**정정:** 보존법칙은 **필요조건**이지 **충분조건**이 아니다.
반응이 실제로 일어나려면 보존법칙 외에도:
- 충분한 **에너지** (문턱 에너지)
- 충분한 **반응률** (반응물의 밀도, 온도)
- 적절한 **시간** (반응이 진행될 만큼)

이 모두 필요하다. I4.2 OX 문제가 바로 이 점을 확인한다.

### 오해 2: "반응률 숫자가 실제 물리값이다"

**정정:** 이 노트북의 rate 값 (1.0, 0.5, 2.0)은 **토이 모델의 임의 설정**이다.
실제 반응률은 온도의 복잡한 함수이며, 핵물리 실험으로 측정된다.
여기서는 **네트워크 구조**와 **흐름 방향**을 이해하는 것이 목적이다.

### 오해 3: "He-4가 많은 이유는 하나뿐이다"

**정정:** He-4가 BBN의 주요 생성물인 이유는 **세 가지가 결합**된 결과다:
1. **안정성** (결합에너지가 높음)
2. **네트워크 구조** (모든 경로가 합류)
3. **제한 반응물** (중성자가 부족)

---

# 3차시 핵심 정리

## 세 가지 큰 질문과 답

### Q1. 병목이 풀린 뒤 He-4가 "빠르게" 만들어지는 이유는 무엇인가?

**A:** 병목이 열리면 D가 쌓이기 시작하고, D에서 출발하는 여러 경로가 **모두 He-4로 합류**한다.
He-4는 핵자당 결합에너지가 가장 높아 **가장 안정**하므로, 도달하면 더 이상 변하지 않는다.

### Q2. 핵반응에서도 무엇이 반드시 보존되는가?

**A:**
1. **질량수(A):** 핵자(양성자+중성자)의 총 개수
2. **전하(Z):** 양성자의 총 개수

### Q3. 여러 반응이 얽힌 네트워크에서 "어느 길이 주요 경로인가"를 어떻게 판단하는가?

**A:**
1. **반응률 비교:** 반응률이 높은 경로가 주요 경로
2. **민감도 분석:** 반응률을 바꿨을 때 결과가 크게 변하는 반응이 중요
3. **흐름 추적:** 시뮬레이션에서 어느 경로로 물질이 많이 흐르는지 관찰

---

## 다음 차시 예고

3차시는 **"He-4까지 어떻게 가는가"**를 다루었습니다.

4차시에서는:
- 질량수 7 경로 (${}^7\text{Be}$, ${}^7\text{Li}$)를 추가합니다
- **"리튬-7 문제"**를 소개합니다 (관측과 이론의 불일치!)
- 과학의 미해결 문제를 토이 모델로 탐구합니다

---

축하합니다! 3차시를 완료했습니다!

In [ ]:
# 전체 채점
grader.check_all()